In [ ]:
# jupyter notebook tested on docker tensorflow/tensorflow:2.4.1-gpu-jupyter

import glob, shutil, os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.layers.experimental import preprocessing
from keras_preprocessing.image import ImageDataGenerator

from pandas import Series, DataFrame
from PIL import Image, ImageEnhance, ImageOps

os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

from tensorflow.keras.mixed_precision import experimental as mixed_precision
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION"] = '1'

np.random.seed(0)
%matplotlib inline

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
        print('{} memory growth: {}'.format(device, tf.config.experimental.get_memory_growth(device)))
else:
    print("Not enough GPU hardware devices available")

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

In [ ]:
### class list ###
annotated_group =['C_01_glomeruli', 
 'C_02_mild_mesangial_change',
 'C_12_crescent',
 'C_03_severe_mesangial_change',
 'C_04_double_contour',
 'C_05_endo_hypercellularity',
 'C_06_vascular_pole',
 'C_09_global_sclerosis_2',
 'C_07_collapsed_glomeruli',
 'C_08_global_sclerosis_1',
 'C_10_bowmans_capsule_1',
 'C_11_bowmans_capsule_2',
 'C_14_cortex_tubules_1',
 'C_16_tubules_with_droplets',
 'C_15_cortex_tubules_2',
 'C_18_atrophic_tubules',
 'C_19_inflammation_1',
 'C_17_foam_cells',
 'C_46_broken',
 'C_22_corticomedulla',
 'C_23_medulla_1',
 'C_24_medulla_2',
 'C_25_medulla_3',
 'C_26_medulla_4',
 'C_27_collecting_duct',
 'C_20_inflammation_2',
 'C_34_artery_1',
 'C_35_artery_2',
 'C_36_vessel',
 'C_28_cast_1',
 'C_31_cast_4',
 'C_29_cast_2',
 'C_30_cast_3',
 'C_32_cast_5',
 'C_33_bleb',
 'C_13_amyloid',
 'C_37_matrix',
 'C_38_muscle',
 'C_39_fat',
 'C_47_unfocused',
 'C_48_prep_failure',
 'C_40_rbc',
 'C_49_dust',
 'C_44_compressed',
 'C_21_inflammation_3',
 'C_41_other_1',
 'C_42_other_2',
 'C_52_red_marker',
 'C_53_green_marker',
 'C_54_blue_marker',
 'C_55_black_marker',
 'C_56_black',
 'C_50_artifact',
 'C_45_edge',
 'C_43_noise',
 'C_51_blank']

In [ ]:
annotated_dict = dict(zip(np.arange(len(annotated_group)), annotated_group))
annotated_dict

In [ ]:
NUM_CLASSES = len(annotated_group)
IMG_SIZE = 456
BATCH_SIZE = 64
WORKERS = 32

#EfficientNetB0 	224
#EfficientNetB1 	240
#EfficientNetB2 	260
#EfficientNetB3 	300
#EfficientNetB4 	380
#EfficientNetB5 	456
#EfficientNetB6 	528
#EfficientNetB7 	600

In [ ]:
def build_model():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB5(include_top=False, input_tensor=inputs, weights=None,
                           drop_connect_rate=0.3)
    model.trainable = False
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.Dropout(0.2, name="top2_dropout")(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.2, name="top1_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax', name="pred")(x)
    
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

with strategy.scope():
    model = build_model()
model.summary()

In [ ]:
# load weight
# set path to the trained weight
model.load_weights(#set path to 'EfficientNetB5_56_classes_trained_wight_CNN_Man.h5')


In [ ]:
# set image_directory 
# assumed directory structure is 'YOUR_TEST_DIRECTORY/SUB_DIRECTORY/IMAGES'

test_dir = # YOUR_TEST_DIRECTORY

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                    target_size=(IMG_SIZE,IMG_SIZE), 
                                                    batch_size=BATCH_SIZE,
                                                    color_mode="rgb",
                                                    shuffle = False,
                                                    class_mode='categorical',
                                                   interpolation = 'bilinear')

In [ ]:
test_filenames = test_generator.filenames
test_filenames

In [ ]:
predicted_scores = model.predict_generator(test_generator,workers=WORKERS, verbose=1)
predicted_labels = np.argmax(predicted_scores, axis=1)
predicted_probabilities = []
for i, j in zip(predicted_scores, predicted_labels):
    predicted_probabilities.append(i[j])

In [ ]:
predicted_scores

In [ ]:
# See ’annotated_dict' for the meanings of predicted_labels
predicted_labels

In [ ]:
predicted_probabilities